<a href="https://colab.research.google.com/github/Nischal2015/ncit-workshop/blob/main/1-basic_agents/2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lab 2
### Building a "Financial Analyst" Agent

### Run this if you are using VSCode

In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))
from core import load_vault_env

load_vault_env()

### Run this if you are using Google Colab

In [ ]:
# !pip install langsmith["openai-agents"] openai-agents langchain langchain-openai tavily-python

In [2]:
# import os
# from google.colab import userdata

# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# os.environ["LANGSMITH_API_KEY"] = userdata.get("LANGSMITH_API_KEY")
# os.environ["LANGSMITH_PROJECT"] = "ncit-workshop"
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"

### Imports

In [3]:
from agents import (
    Agent,
    Runner,
    FunctionTool,
    ModelSettings,
    function_tool,
    set_trace_processors,
    trace,
)
from langsmith.wrappers import OpenAIAgentsTracingProcessor
import nest_asyncio

nest_asyncio.apply()
set_trace_processors([OpenAIAgentsTracingProcessor(name="Financial Agents Tracer")])

### Defining Tools

Tools provides Agents (LLM) with the following capabilities:
- Interact with external systems, access real-time information, and perform actions beyond text generation.
- Call APIs, execute code, retrieve data, and automate complex workflows.
- Extend their functionality beyond reasoning and language generation.

In [4]:
@function_tool
def calculate_mortgage(principal: float, rate: float, years: int) -> float:
    """Calculates the monthly mortgage payment.

    Args:
        principal (float): The loan principal amount.
        rate (float): The annual interest rate (in decimals).
        years (int): The loan term in years.

    Returns:
        float: The monthly mortgage payment rounded to two decimal places.
    """
    monthly_rate = rate / 100 / 12
    payments = years * 12
    if monthly_rate == 0:
        return principal / payments
    payment = (principal * monthly_rate) / (1 - (1 + monthly_rate) ** -payments)
    return round(payment, 2)


@function_tool
def get_interest_rates(region: str) -> str:
    """Returns the current mortgage interest rates for a given region.

    Args:
        region (str): The region to get interest rates for.

    Returns:
        str: A string describing the current interest rates.
    """
    rates = {
        "US": "The current average mortgage interest rate in the US is around 3.5% for a 30-year fixed loan.",
        "Nepal": "The current average mortgage interest rate in Nepal is around 9% for a 20-year fixed loan.",
    }
    return rates.get(region, "Interest rate information not available for this region.")

### Defining Financial Analyst Agent

In [5]:
analyst_agent = Agent(
    name="Mortgage Analyst Agent",
    model="gpt-4.1-mini",
    instructions=(
        "You are a helpful finanical analyst agent that helps users with mortgage calculations and interest rate information."
    ),
    tools=[calculate_mortgage, get_interest_rates],
)

In [6]:
import json

for tool in analyst_agent.tools:
    if isinstance(tool, FunctionTool):
        print(f"Tool Name: {tool.name}")
        print(f"Description: {tool.description}")
        print(f"Json schema: {json.dumps(tool.params_json_schema, indent=2)}\n")

Tool Name: calculate_mortgage
Description: Calculates the monthly mortgage payment.
Json schema: {
  "properties": {
    "principal": {
      "description": "The loan principal amount.",
      "title": "Principal",
      "type": "number"
    },
    "rate": {
      "description": "The annual interest rate (in decimals).",
      "title": "Rate",
      "type": "number"
    },
    "years": {
      "description": "The loan term in years.",
      "title": "Years",
      "type": "integer"
    }
  },
  "required": [
    "principal",
    "rate",
    "years"
  ],
  "title": "calculate_mortgage_args",
  "type": "object",
  "additionalProperties": false
}

Tool Name: get_interest_rates
Description: Returns the current mortgage interest rates for a given region.
Json schema: {
  "properties": {
    "region": {
      "description": "The region to get interest rates for.",
      "title": "Region",
      "type": "string"
    }
  },
  "required": [
    "region"
  ],
  "title": "get_interest_rates_args"

In [7]:
result = await Runner.run(
    starting_agent=analyst_agent,
    input="What would my payment be for a $500k house in the US for 30 years?",
)
print(result.final_output)

For a $500,000 house in the US with a 30-year mortgage at an interest rate of 3.5%, your estimated monthly payment would be approximately $1,396.21. Let me know if you need more details or other calculations!


### Using external API as tool

In [ ]:
import os
from tavily import TavilyClient

tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [ ]:
@function_tool
def tavily_search(query: str) -> str:
    """
    Perform a web search using Tavily and return a summarized result.

    Args:
        query (str): The search query.

    Returns:
        str: A summary of the top search results.
    """
    response = tavily_client.search(query, search_depth="advanced", max_results=3)
    results = response.get("results", [])
    return results or "No results found."

In [ ]:
from datetime import date

current_date = date.today().strftime("%B %d, %Y")

# Create an agent that uses the Tavily search tool
search_agent = Agent(
    name="Web Research Agent",
    model="gpt-4.1",
    instructions=f"""
    You are a web research agent that uses the Tavily tool to find information on the web.
    Today's date is {current_date}
    """,
    tools=[tavily_search],
    model_settings=ModelSettings(temperature=0.4, tool_choice="required"),
)

In [ ]:
set_trace_processors([OpenAIAgentsTracingProcessor(name="Web Research Agent Tracer")])

with trace(
    workflow_name="Web Research Workflow",
    metadata={
        "project": "ncit-workshop",
        "description": "An agent that performs web research using Tavily.",
    },
):
    result = await Runner.run(
        starting_agent=search_agent,
        input="What is the current state of world economics with the advent of AI technologies? I want to know about massive economics shifts, acquisitions, and market changes.",
    )
    print(result.final_output)